MOBILE NETS are lighter and faster in performance as compared to other deep conv nets like vgg but there is a tradeoff of accuracy. Mobilenets aren't that accurate as compared.

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir("drive/DogBreedClassification/")

In [0]:
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
import itertools


In [0]:
mobile = keras.applications.mobilenet.MobileNet()

In [0]:
mobile.summary()


In [0]:
training_set = ImageDataGenerator(preprocessing_function = keras.applications.mobilenet.preprocess_input ).flow_from_directory('train',
                                                 target_size = (224, 224),
                                                 color_mode= 'rgb',
                                                 batch_size = 100)  

In [0]:
x = mobile.layers[-6].output
predictions = Dense(120,activation='softmax')(x)
model = Model(inputs= mobile.input, outputs= predictions)
model.summary()

In [0]:
for layer in model.layers[:-6]: 
    layer.trainable = False

In [0]:
from keras import optimizers
adam = optimizers.Adam(lr = 1e-4)
model.compile(adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit_generator(training_set,
                         samples_per_epoch = 100000,
                         nb_epoch = 10)


model.save('model1_mobilenet.h5')